In [1]:
from matplotlib import pyplot as plt
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
from mediapipe import solutions
import cv2 as cv
import numpy as np
import time

In [2]:
RPS_MARGIN = 40
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

In [3]:
plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'xtick.labelbottom': False,
    'xtick.bottom': False,
    'ytick.labelleft': False,
    'ytick.left': False,
    'xtick.labeltop': False,
    'xtick.top': False,
    'ytick.labelright': False,
    'ytick.right': False
})

mp_hands = solutions.hands
mp_drawing = solutions.drawing_utils
mp_drawing_styles = solutions.drawing_styles

In [4]:
def draw_landmarks_on_image(rgb_image, detection_result, result):
    hand_landmarks_list = detection_result.hand_landmarks
    annotated_image = np.copy(rgb_image)
    
    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
              annotated_image,
              hand_landmarks_proto,
              solutions.hands.HAND_CONNECTIONS,
              solutions.drawing_styles.get_default_hand_landmarks_style(),
              solutions.drawing_styles.get_default_hand_connections_style())
    
        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
                
        # Draw rock, paper or scissor
        text_x = int(min(x_coordinates) * width) 
        text_y = int(min(y_coordinates) * height) - RPS_MARGIN
        cv.putText(annotated_image, f"{result[0][0].category_name}",
                    (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)
    
    return annotated_image

In [5]:
# STEP 2: Create an GestureRecognizer object.
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a gesture recognizer instance with the video mode:
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='model/exported_model_no_flip/gesture_recognizer.task'),
    # base_options=BaseOptions(model_asset_path='model/exported_model/gesture_recognizer.task'),
    running_mode=VisionRunningMode.VIDEO)

In [6]:
cam = cv.VideoCapture(0) 
cv.namedWindow("Cam")    

with GestureRecognizer.create_from_options(options) as recognizer:
    while cam.isOpened():
            
        _, frame = cam.read()
        cv.imshow("Cam", frame)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        gesture_recognition_result = recognizer.recognize_for_video(mp_image, int(round(time.time() * 100)))
        annotated_image = mp_image.numpy_view()

        if len(gesture_recognition_result.gestures)>0:
            top_gesture = gesture_recognition_result.gestures[0][0]
            hand_landmarks = gesture_recognition_result.hand_landmarks
            result = [(top_gesture,hand_landmarks)]
            annotated_image = draw_landmarks_on_image(annotated_image, gesture_recognition_result, result)

        cv.imshow("Salida", annotated_image)

        if cv.waitKey(10) & 0xFF == 27: cam.release()
    
    cv.destroyAllWindows()